In [ ]:
# 必要なライブラリのインストールとインポート
!pip install compressai

import torch
import matplotlib.pyplot as plt
from compressai.zoo import mbt2018

print("✅ セットアップ完了！")


In [ ]:
# 訓練済みモデルの読み込み
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔍 使用デバイス: {device}")

# CompressAIライブラリのバージョン確認
import compressai
print(f"🔍 CompressAI version: {compressai.__version__}")

try:
    model = mbt2018(quality=3, pretrained=True, progress=True)
    model = model.to(device)
    model.eval()
    print(f"✅ 訓練済みモデル読み込み完了")
    
    # モデルの基本情報を確認
    total_params = sum(p.numel() for p in model.parameters())
    print(f"🔍 モデルパラメータ数: {total_params:,}")
    
except Exception as e:
    print(f"❌ モデル読み込みエラー: {e}")
    raise


In [ ]:
# シンプルなテスト画像を作成
test_image = torch.zeros(1, 3, 256, 256, dtype=torch.float32).to(device)
test_image[0, 0, :128, :128] = 1.0  # 左上: 赤
test_image[0, 1, :128, 128:] = 1.0  # 右上: 緑
test_image[0, 2, 128:, :128] = 1.0  # 左下: 青
test_image[0, :, 128:, 128:] = 0.5  # 右下: グレー

print("✅ テスト画像作成完了")
print(f"🔍 画像shape: {test_image.shape}")
print(f"🔍 画像dtype: {test_image.dtype}")
print(f"🔍 画像device: {test_image.device}")
print(f"🔍 画像値の範囲: [{test_image.min().item():.3f}, {test_image.max().item():.3f}]")
print(f"🔍 画像サイズ: {test_image.numel() * 4} bytes")


In [ ]:
# 画像圧縮・展開の実行
print("🔄 圧縮・展開を開始...")

try:
    with torch.no_grad():
        # 圧縮前の詳細確認
        print(f"🔍 圧縮前画像の詳細確認:")
        print(f"   - Shape: {test_image.shape}")
        print(f"   - Device: {test_image.device}")
        print(f"   - Model device: {next(model.parameters()).device}")
        
        # デバイス同期の確認
        if test_image.device != next(model.parameters()).device:
            print("⚠️ デバイス不一致を検出、修正中...")
            test_image = test_image.to(next(model.parameters()).device)
        
        # 圧縮
        print("🗜️ 圧縮中...")
        compressed = model.compress(test_image)
        print(f"✅ 圧縮完了")
        
        # 展開
        print("📤 展開中...")
        decompressed = model.decompress(compressed["strings"], compressed["shape"])
        reconstructed = decompressed["x_hat"]
        print(f"✅ 展開完了")
        
        print(f"🔍 復元画像shape: {reconstructed.shape}")
        print(f"🔍 復元画像device: {reconstructed.device}")

except Exception as e:
    print(f"❌ 圧縮・展開エラー: {e}")
    import traceback
    traceback.print_exc()
    raise


In [ ]:
# 🎯 最終結果の表示
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 元画像
original_img = test_image[0].cpu().permute(1, 2, 0).clamp(0, 1)
axes[0].imshow(original_img)
axes[0].set_title('元画像', fontsize=16, fontweight='bold')
axes[0].axis('off')

# 圧縮・復元後の画像
reconstructed_img = reconstructed[0].cpu().permute(1, 2, 0).clamp(0, 1)
axes[1].imshow(reconstructed_img)
axes[1].set_title('圧縮・復元後', fontsize=16, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# 🔍 デバッグ情報を追加
print("🔍 === デバッグ情報 ===")
print(f"compressed keys: {list(compressed.keys())}")
print(f"strings length: {len(compressed['strings'])}")
for i, s in enumerate(compressed['strings']):
    print(f"string[{i}] length: {len(s)} bytes")
print(f"shape: {compressed['shape']}")

# 圧縮効果の数値（修正版）
original_size_bytes = test_image.numel() * 4  # float32 = 4 bytes per element
compressed_size_bytes = sum(len(s) for s in compressed['strings'])  # 全stringの合計
compression_ratio = original_size_bytes / compressed_size_bytes if compressed_size_bytes > 0 else 0
mse = torch.mean((test_image - reconstructed) ** 2).item()
psnr = -10 * torch.log10(torch.tensor(mse)) if mse > 0 else float('inf')

print("\n🎯 === 修正版結果 ===")
print(f"📊 圧縮率: {compression_ratio:.1f}倍")
print(f"📈 PSNR: {psnr:.1f} dB")
print(f"💾 元サイズ: {original_size_bytes:,} bytes")
print(f"🗜️ 圧縮後: {compressed_size_bytes:,} bytes")
print(f"📏 画像サイズ: {test_image.shape}")
print(f"🔢 MSE: {mse:.6f}")

if compressed_size_bytes > 0 and 10 <= compression_ratio <= 200 and psnr >= 20:
    print("\n✅ 学習ベース画像圧縮の成功！")
else:
    print("\n⚠️ 異常な結果が検出されました。モデルまたはデータに問題がある可能性があります。")
